# Laboratorio 8 
    - Francis Aguilar - 22243
    - César López - 22535
    - Angela García -22869 


enlace al repositorio: https://github.com/angelargd8/lab8-ds


In [0]:

# Cargar el archivos
# /Volumes/workspace/default/data_lab8/data lab8/fallecidos/2013 vehiculos involucrados.xlsx
path_fallecidos = "/Volumes/workspace/default/data_lab8/data lab8/fallecidos/"
path_hechos = "/Volumes/workspace/default/data_lab8/data lab8/hechos y transito/"
path_vehiculos = "/Volumes/workspace/default/data_lab8/data lab8/vehiculos involucrados/"

In [0]:
# %pip install openpyxl
# %pip install unidecode

### Limpieza de datos


In [0]:
import pandas as pd
import glob
import os

paths = [path_fallecidos, path_hechos, path_vehiculos]
# paths = [path_fallecidos]

# convertimos los datos a csv antes de leerlos con spark 
for path in paths: 
    for file in glob.glob(os.path.join(path, "*.xlsx")):
        df = pd.read_excel(file)
        csv_path = file.replace(".xlsx", ".csv")
        df.to_csv(csv_path, index=False)



In [0]:

# Listar los CSVs en la carpeta
files_fallecidos = [f.path for f in dbutils.fs.ls(path_fallecidos) if f.name.endswith(".csv")]
files_hechos = [f.path for f in dbutils.fs.ls(path_hechos) if f.name.endswith(".csv")]
files_vehiculos = [f.path for f in dbutils.fs.ls(path_vehiculos) if f.name.endswith(".csv")]


def mostrar_columnas(files):
    columnas_unicas = set()

    for file in files:
        try:
            df = spark.read.option("header", "true").csv(file, inferSchema=True)
            count = df.count()
            print(f"ARCHIVO: {file}")
            print(f"   Columnas ({len(df.columns)}): {df.columns}")
            print(f"   Registros: {count}")
            print("-" * 80)

            # Agregar columnas al conjunto (evita duplicados)
            columnas_unicas.update(df.columns)

        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    # Convertir a lista ordenada alfabéticamente
    columnas_ordenadas = sorted(columnas_unicas)
    
    print("\nLista de todas las columnas únicas (ordenadas alfabéticamente):")
    for col in columnas_ordenadas:
        print(f" - {col}")


In [0]:
# visualizamos las columnas antes de realizar cambios 
mostrar_columnas(files_fallecidos)

In [0]:
import os
import unidecode
from pyspark.sql import functions as F
import re 

archivos = []

columnas_finales = [
    'num_correlativo', 'anio_ocu', 'mes_ocu', 'dia_ocu', 'dia_sem_ocu',
    'hora_ocu', 'depto_ocu', 'mupio_ocu', 'zona_ocu', 'sexo_per',
    'edad_per', 'mayor_menor', 'tipo_veh', 'marca_veh', 'color_veh','edad_pil',  'sexo_pil','edad_vic',
    'modelo_veh', 'tipo_eve', 'fallecidos_lesionados', 'archivo_origen'
]

columnas_a_eliminar = [
            "g_edad_2", "g_edad_60ymás", "g_edad_80ymás",
            "Otro_g_edad_fall_les", "edad_quinquenales",
            "g_hora_5", "g_modelo_veh", "int_o_noint",
            "área_geo_ocu", "g_hora", "corre_base"
        ]
# --- Renombrar variaciones comunes ---
renombres = {
            "Núm_corre": "num_correlativo",
            "núm_corre": "num_correlativo",
            "num_corre": "num_correlativo",
            "num_hecho": "num_correlativo",
            "fall_les": "fallecidos_lesionados",
            "edad_pil": "edad_per",
            "sexo_pil": "sexo_per",
            "sexo_vic": "sexo_per",
            "edad_vic": "edad_per",
            "ano_ocu": "anio_ocu",  
            "año_ocu": "anio_ocu" , 
            "área_geo_ocu":"areag_ocu"
        }

def convert_csv(files, path, columnas_a_eliminar, renombres):
    for file in files:
        df = spark.read.option("header", "true").csv(file, inferSchema=True)
        if file == "dbfs:/Volumes/workspace/default/data_lab8/data lab8/fallecidos/2014 fallecidos y lesionados.csv" or file == "dbfs:/Volumes/workspace/default/data_lab8/data lab8/hechos y transito/2014 hechos de transito.csv": 
            df = df.drop("num_corre", "corre_base") 
        
        for col in columnas_a_eliminar:
            if col in df.columns:
                df = df.drop(col)


        for old, new in renombres.items():
            if old in df.columns:
                df = df.withColumnRenamed(old, new)

        # --- Quitar tildes a todas las columnas ---
        df = df.toDF(*[unidecode.unidecode(c).lower() for c in df.columns])

        # --- Agregar columna de origen ---
        nombre_archivo = os.path.basename(file)
        df = df.withColumn("archivo_origen", F.lit(nombre_archivo))


        # --- extraer el año del nombre del archivo ---
        match = re.search(r'(\d{4})', nombre_archivo)
        anio_archivo = match.group(1) if match else "Ignorado"

        # --- Rellenar columnas faltantes con "Ignorado" ---
        for col in columnas_finales:
            if col not in df.columns:
                if col == "anio_ocu":
                    df = df.withColumn(col, F.lit(anio_archivo))  # usar año extraído
                else:
                    df = df.withColumn(col, F.lit("Ignorado"))

        # --- Reordenar columnas ---
        cols_presentes = [c for c in columnas_finales if c in df.columns]
        df = df.select(*cols_presentes)

        # --- Guardar ---
        nombre_salida = os.path.splitext(nombre_archivo)[0] + "_limpio.csv"
        ruta_salida = os.path.join(path + 'data_clean/', nombre_salida)
        df.coalesce(1).write.mode("overwrite").option("header", "true").csv(ruta_salida)

        archivos.append(df)
        print(f"\nProcesado: {nombre_archivo}")
        print("Columnas finales:", df.columns)

convert_csv(files_fallecidos, path_fallecidos, columnas_a_eliminar, renombres)


In [0]:
def leer_df(path):
    df = (
        spark.read
        .option("header", True)
        .option("delimiter", ",")
        .option("recursiveFileLookup", "true")  #  busca dentro de subcarpetas
        .csv(path + "data_clean/")
    )
    df.printSchema()   # Muestra las columnas y tipos de datos
    total_filas = df.count()   # Cuenta las filas

    print(f"El DataFrame tiene {total_filas} registros.")
    print(len(df.columns))
    return df

df_fallecidos= leer_df(path_fallecidos)


In [0]:
#Revisión de valores únicos y duplicados
for c in df_fallecidos.columns:
    print(c, "→", df_fallecidos.select(c).distinct().count())

In [0]:
def mostrar_contenido(df):
    for c in df.columns:
        print(f"\n=============== Columna: {c} ===============")
        valores = df.select(c).distinct().collect()  
        print([v[c] for v in valores])

mostrar_contenido(df_fallecidos)

In [0]:
import pyspark.sql.functions as F
cols_to_int = [ "edad_per", "edad_pil"]

# Definir todos los mapeos
mapeos = {
    "mes_ocu": {
        "1": "Enero", "2": "Febrero", "3": "Marzo", "4": "Abril", "5": "Mayo",
        "6": "Junio", "7": "Julio", "8": "Agosto", "9": "Septiembre",
        "10": "Octubre", "11": "Noviembre", "12": "Diciembre"
    },
    "dia_sem_ocu": {
        '1': 'Lunes', '2': 'Martes', '3': 'Miércoles',
        '4': 'Jueves', '5': 'Viernes', '6': 'Sábado', '7': 'Domingo',
        'Sabado': 'Sábado', 'Miercoles': 'Miércoles'
    },
    "sexo_per": {
        '1': 'Hombre',
        '2': 'Mujer',
        '9': 'Ignorado',
    },
    "tipo_veh": {
        '1': 'Automovil', '2': 'Camioneta', '3': 'Pick up', '4': 'Motocicleta',
        '5': 'Camión', '6': 'Cabezal', '7': 'Bus extraurbano', '8': 'Jeep',
        '9': 'Microbús', '10': 'Taxi', '11': 'Panel', '12': 'Bus urbano',
        '13': 'Tractor', '14': 'Mototaxi', '15': 'Furgón', '16': 'Grúa',
        '17': 'Bus escolar', '18': 'Bicicleta', '99': 'Ignorado', '19':'Avioneta',
        '20':'Montacargas', '21':'Bus militar', '22':'Cuatrimoto', '23':'Furgoneta','Microb?s':'Microbús',
        'Cami?n':'Camión', 'Motos acu?ticas':'Motos acuáticas', 'Gr?a':'Grúa','Autom?vil':'Automóvil'
    },
    "mayor_menor": {
        '1': 'Mayor',
        '2': 'Menor',
        '9':'Ignorado'
    },
    "fallecidos_lesionados": {
        '1': 'Fallecido',
        '2': 'Lesionado',
        '9': 'Ignorado'
    },
    "tipo_eve": {
        '1': 'Colisión', '2': 'Choque', '3': 'Vuelco', '4': 'Caída',
        '5': 'Atropello', '6': 'Derrape', '7': 'Embarranco',
        '8': 'Encuneto', '99': 'Ignorado'
    },
    "color_veh": {
        '1': 'Rojo', '2': 'Blanco', '3': 'Azul', '4': 'Gris', '5': 'Negro',
        '6': 'Verde', '7': 'Amarillo', '8': 'Celeste', '9': 'Corinto',
        '10': 'Café', '11': 'Beige', '12': 'Turquesa', '13': 'Marfil',
        '14': 'Anaranjado', '15': 'Morado', '16': 'Rosado',
        '17': 'Varios colores', '99': 'Ignorado','Colisi?n': 'Colisión',
        'Ca?da': 'Caída',
        'Encunet?': 'Encuneto',
        'Embarranc?':'Embarranco',
        'Embarrancó':'Embarranco',
    },
    "edad_per": {
        'Ignorado': '999',
        'Ignorada': '999',
        
    },
    "edad_pil": {
        'Ignorado': '999',
        'Ignorada': '999',
        
    },
    "zona_ocu": {
        'Ignorado': '99',
        'Ignorada': '99',
    },
     "hora_ocu": {
        'Ignorado': '99',
        'Ignorada':'99'
    },
      "areag_ocu": {
        '1': 'Urbana',
        '2': 'Rural',
    },
      "estado_pil": {
        '1': 'Normal',
        '2': 'Bajo efectos de alcohol',
        '3':'Drogado', 
        '9':'Ignorado'
    },
      "sexo_pil": {
        '1': 'Hombre',
        '2': 'Mujer',
        '9': 'Ignorado',
    },
     
     
    
}

# Función para aplicar mapeos de forma eficiente
def aplicar_mapeo(df, columna, mapeo_dict):
    """Aplica un mapeo a una columna del DataFrame"""
    map_expr = F.create_map([F.lit(x) for kv in mapeo_dict.items() for x in kv])
    return df.withColumn(columna, F.coalesce(map_expr[F.col(columna)], F.col(columna)))


# Leer municipios una sola vez
dptos_muni = spark.read.option("header", "true").csv("/Volumes/workspace/default/data_lab8/data lab8/municipios.csv")
dic_municipios = {row['numero']: row['departamento'] for row in dptos_muni.collect()}

# Leer marcas de carros
marcas = spark.read.option("header", "true").csv("/Volumes/workspace/default/data_lab8/data lab8/marcas.csv")
dic_marcas = {row['numero']: row['marca'] for row in marcas.collect()}

# Agregar mapeo de municipios
mapeos["depto_ocu"] = dic_municipios
mapeos["mupio_ocu"] = dic_municipios
mapeos["marca_veh"] = dic_marcas

# Aplicar todos los mapeos en un solo loop
for columna, mapeo in mapeos.items():
    if columna in df_fallecidos.columns:
        df_fallecidos = aplicar_mapeo(df_fallecidos, columna, mapeo)

# Convertir columnas a integer
for col in cols_to_int:
    if col in df_fallecidos.columns:
        df_fallecidos = df_fallecidos.withColumn(col, F.col(col).cast("int"))

print(df_fallecidos.columns)

In [0]:
mostrar_contenido(df_fallecidos)


In [0]:
mostrar_columnas(files_hechos)

In [0]:
columnas_finales = [
    'num_correlativo', 'anio_ocu', 'mes_ocu', 'dia_ocu', 'dia_sem_ocu',
    'hora_ocu', 'depto_ocu', 'mupio_ocu', 'zona_ocu', 'sexo_per','causa_acc',
    'edad_per','edad_pil',  'sexo_pil','estado_pil','mayor_menor', 'tipo_veh', 'marca_veh', 'color_veh',
    'modelo_veh', 'tipo_eve', 'fallecidos_lesionados','areag_ocu', 'archivo_origen']


columnas_a_eliminar = [
            "g_edad_2", "g_edad_60ymás", "g_edad_80ymás",
            "Otro_g_edad_fall_les", "edad_quinquenales",
            "g_hora_5", "g_modelo_veh", "int_o_noint",
            "área_geo_ocu", "g_hora", "corre_base"
        ]
renombres = {
            "Año_ocu":"anio_ocu",
            "edad_con":"edad_pil",
            "sexo_con":"sexo_pil",
            "causa_acc":"tipo_eve",
            "estado_con":"estado_pil",
            "Núm_corre": "num_correlativo",
            "núm_corre": "num_correlativo",
            "num_corre": "num_correlativo",
            "num_hecho": "num_correlativo",
            "fall_les": "fallecidos_lesionados",
            "sexo_vic": "sexo_per",
            "edad_vic": "edad_per",
            "ano_ocu": "anio_ocu",  
            "año_ocu": "anio_ocu"    
        }
convert_csv(files_hechos, path_hechos, columnas_a_eliminar, renombres)

In [0]:
df_hechos = leer_df(path_hechos)

In [0]:
mostrar_contenido(df_hechos)

In [0]:

# Aplicar todos los mapeos en un solo loop
for columna, mapeo in mapeos.items():
    if columna in df_hechos.columns:
        df_hechos = aplicar_mapeo(df_hechos, columna, mapeo)

for col in cols_to_int:
    if col in df_hechos.columns:
        df_hechos = df_hechos.withColumn(col, F.col(col).cast("int"))

print(df_hechos.columns)

In [0]:

mostrar_contenido(df_hechos)

In [0]:
mostrar_columnas(files_vehiculos)

In [0]:
columnas_finales = [
    'num_correlativo', 'anio_ocu', 'mes_ocu', 'dia_ocu', 'dia_sem_ocu',
    'hora_ocu', 'depto_ocu', 'mupio_ocu', 'zona_ocu', 'sexo_per','causa_acc',
    'edad_per','edad_pil',  'sexo_pil','estado_pil','mayor_menor', 'tipo_veh', 'marca_veh', 'color_veh',
    'modelo_veh', 'tipo_eve', 'fallecidos_lesionados','areag_ocu', 'archivo_origen']


columnas_a_eliminar = [
            "g_edad_2", "g_edad_60ymás", "g_edad_80ymás",
            "Otro_g_edad_fall_les", "edad_quinquenales",
            "g_hora_5", "g_modelo_veh", "int_o_noint",
            "área_geo_ocu", "g_hora", "corre_base"
        ]
renombres = {
            "Año_ocu":"anio_ocu",
            "edad_con":"edad_pil",
            "sexo_con":"sexo_pil",
            "causa_acc":"tipo_eve",
            "estado_con":"estado_pil",
            "Núm_corre": "num_correlativo",
            "núm_corre": "num_correlativo",
            "num_corre": "num_correlativo",
            "num_hecho": "num_correlativo",
            "fall_les": "fallecidos_lesionados",
            "sexo_vic": "sexo_per",
            "edad_vic": "edad_per",
            "ano_ocu": "anio_ocu",  
            "año_ocu": "anio_ocu"    
        }
convert_csv(files_vehiculos, path_vehiculos, columnas_a_eliminar, renombres)

In [0]:
df_vehiculos = leer_df(path_vehiculos)


In [0]:
mostrar_contenido(df_vehiculos)

In [0]:
for columna, mapeo in mapeos.items():
    if columna in df_vehiculos.columns:
        df_vehiculos = aplicar_mapeo(df_vehiculos, columna, mapeo)
        
for col in cols_to_int:
    if col in df_vehiculos.columns:
        df_vehiculos = df_vehiculos.withColumn(col, F.col(col).cast("int"))
print(df_vehiculos.columns)

In [0]:
mostrar_contenido(df_vehiculos)

### 1. Mostrar cuántos registros hay en cada tabla (hechos, vehículos, fallecidos, lesionados). Muestre algunos resultados con la función .show(). Genere un describe y summary para aquellas columnas que considere importantes según cada archivo.


In [0]:
print("Registros en cada tabla:")
print(f"Hechos: {df_hechos.count()}")
print(f"Vehículos: {df_vehiculos.count()}")
print(f"Fallecidos: {df_fallecidos.count()}")

In [0]:
df_fallecidos.show(5)

In [0]:
df_fallecidos.describe(["fallecidos_lesionados"]).show()
df_fallecidos.select("dia_ocu", "edad_per").summary().show()


In [0]:
df_hechos.show(5)

In [0]:
df_hechos.describe(["tipo_eve"]).show()
df_hechos.select("dia_ocu", "edad_per").summary().show()


In [0]:
df_vehiculos.show(5)


In [0]:
df_vehiculos.describe(["tipo_veh"]).show()
df_vehiculos.select("dia_ocu", "edad_pil").summary().show()


### 2. Identificar los años disponibles en cada tabla y validar si coinciden. 

In [0]:
print("=== Años en HECHOS ===")
df_hechos.select("anio_ocu").distinct().orderBy("anio_ocu").show()

print("=== Años en VEHÍCULOS ===")
df_vehiculos.select("anio_ocu").distinct().orderBy("anio_ocu").show()

print("=== Años en FALLECIDOS ===")
df_fallecidos.select("anio_ocu").distinct().orderBy("anio_ocu").show()

In [0]:
# Convertir los resultados a listas de años
anios_hechos = [row["anio_ocu"] for row in df_hechos.select("anio_ocu").distinct().collect()]
anios_vehiculos = [row["anio_ocu"] for row in df_vehiculos.select("anio_ocu").distinct().collect()]
anios_fallecidos = [row["anio_ocu"] for row in df_fallecidos.select("anio_ocu").distinct().collect()]

# Comparar conjuntos
set_hechos = set(anios_hechos)
set_vehiculos = set(anios_vehiculos)
set_fallecidos = set(anios_fallecidos)

print("Años en hechos:", set_hechos)
print("Años en vehículos:", set_vehiculos)
print("Años en fallecidos:", set_fallecidos)

# Verificar si todos coinciden
if set_hechos == set_vehiculos == set_fallecidos :
    print("-------- Todos los conjuntos de años coinciden en las tablas.")
else:
    print("Los años NO coinciden entre las tablas.")
    print("Años faltantes o diferentes por tabla:")
    print(" - En vehículos no están:", set_hechos - set_vehiculos)
    print(" - En fallecidos no están:", set_hechos - set_fallecidos)


### 3.  Mostrar los valores distintos de tipo de accidente.

In [0]:

df_fallecidos.select("tipo_eve").distinct().show(truncate=False)

### 4.  Calcular cuántos departamentos únicos aparecen en las bases. 

In [0]:
num_departamentos = df_fallecidos.select("depto_ocu").distinct().count()
print(f"Número de departamentos únicos: {num_departamentos}")

departamentos_unicos = [row['depto_ocu'] for row in df_fallecidos.select("depto_ocu").distinct().collect()]
print(departamentos_unicos)


###  5. ¿Cuál es el total de accidentes por año y departamento? Apóyese de la función groupBy. Investigue la función display que tiene Databricks y muestre su resultado en formato de gráfico de barras. 

In [0]:
# Agrupar por año y departamento y contar accidentes
accidentes_por_anio_depto = df_fallecidos.groupBy("anio_ocu", "depto_ocu").count()

# Mostrar el resultado como tabla
accidentes_por_anio_depto.show()

display(accidentes_por_anio_depto)


Databricks visualization. Run in Databricks to view.

###  6. ¿Qué día de la semana registra más accidentes en 2023? Graficar con display en un gráfico de columnas
Cambiamos la pregunta para que sea al 2023

In [0]:
df_2023 = df_fallecidos.filter(df_fallecidos.anio_ocu == "2023")

# Agrupar por día de la semana y contar accidentes
accidentes_por_dia = df_2023.groupBy("dia_sem_ocu").count()

# Mostrar el resultado como tabla
accidentes_por_dia.show()

# En Databricks, usar display para graficar
display(accidentes_por_dia)


Databricks visualization. Run in Databricks to view.

En 2023 el dia que muestra mas accidentes es el dia domingo

###  7. Mostrar la distribución de accidentes por hora del día en el municipio de Guatemala. Graficar en un histograma. 

In [0]:
from pyspark.sql.functions import col
# Filtrar solo registros del municipio de Guatemala y excluir hora = 99
df_guatemala = df_hechos.filter((col("depto_ocu") == "Guatemala") & (col("hora_ocu") != 99))

# Asegurarse de que la hora sea numérica
df_guatemala = df_guatemala.withColumn("hora_ocu", col("hora_ocu").cast("int"))

# Agrupar por hora y contar accidentes
accidentes_por_hora = df_guatemala.groupBy("hora_ocu").count().orderBy("hora_ocu")

# Mostrar resultados
display(accidentes_por_hora)

Databricks visualization. Run in Databricks to view.

### 8. Unir la tabla de hechos de tránsito con la de vehículos usando una llave compuesta por año, mes, departamento y tipo de accidente. ¿Cuántos registros combinados se logran?

In [0]:
# Realizar la unión usando la llave compuesta
df_combinado = df_hechos.join(
    df_vehiculos,
    on=[
        "anio_ocu",
        "mes_ocu",
        "depto_ocu",
        "tipo_eve"
    ],
    how="inner" 
)

# Contar cuántos registros combinados se logran
total_registros = df_combinado.count()
print(f"Total de registros combinados: {total_registros}")


### 9.  De la unión anterior, calcular el promedio de vehículos por accidente en cada departamento. Guardar este resultado en formato Parquet. Luego, vuelva a cargarlo y grafique los 10 departamentos con más vehículos/accidente.

In [0]:
##

### 10.  Encontrar el top 5 de colores de vehículos más involucrados en accidentes. 

### 11. Calcular cuántos lesionados por atropello hubo en 2024, por mes. Graficar en serie temporal (línea). 


### 12. Relacionar accidentes con fallecidos usando llaves (año, mes, departamento, tipo de accidente). Calcular el total de fallecidos por cada tipo de accidente. Graficar en barras horizontales.

### 13. Usar withColumn para clasificar accidentes en franjas horarias: Mañana [6-12), Tarde [12-18), Noche [18-24), Madrugada [0-6). Mostrar cuántos accidentes ocurren en cada franja. 

### 14.  Calcular el ratio de fallecidos por accidente en cada departamento (fallecidos / accidentes). Guardar el resultado en Parquet. 

### 15. Identificar los grupos de edad más afectados en fallecidos y lesionados. Graficar en barras que permitan comparar a ambos grupos.  

### 16. Calcular, para el municipio de Guatemala, cuántos accidentes hay por zona y cuántos fallecidos se reportan en cada una. Generar un gráfico de barras con ambos indicadores. 

### 17. Crear un DataFrame que muestre el porcentaje de accidentes donde el conductor era hombre vs mujer (tabla vehículos). Guardar como Parquet. Finalmente, vuélvalo a cargar y grafique con display en gráfico de pie